In [69]:
import os #for data load
import numpy as np
import random
from keras.preprocessing.sequence import pad_sequences #for padding data
from keras.utils import to_categorical # for categorizing data
from sklearn.model_selection import train_test_split #for train and test spliting
from keras.models import Sequential #for NN model
from keras.layers import LSTM, Embedding, Dense, TimeDistributed #for NN model
from scienceie_loader import load_tokenized_data, load_data_with_char_offsets #for data preparing also customize dataset requirments for load

In [28]:
#dataset loading from data folder
data_root = 'data'
data_train = os.path.join(data_root, 'train2')
data_test = os.path.join(data_root, 'semeval_articles_test')

In [29]:
#spliting into train and test dataset and with customize function that comes with dataset
# and this function define in scienceie_loader.py
train_docs, train_rels, _ = load_tokenized_data(data_train)
test_docs, test_rels, _ = load_tokenized_data(data_test)

Before feed into neural network we've to follow some for preparing data
1. create unique words and labels
2. separate and creating a new list with words and labels

In [30]:
words = set()
tags = set()
sens=[]
labels=[]
for doc in train_docs:#spiliting doc into a sent 
    s_tokens=[]
    l_tokens=[]
    for token, tag in doc: #spilitng sent into a token
        words.add(token)
        tags.add(tag)
        s_tokens.append(token)
        l_tokens.append(tag)
    sens.append(s_tokens)
    labels.append(l_tokens)

#converting unique words and tags set() to list()
words=list(words)
tags=list(tags)    

print("Sentence and labels:\n")
print(sens[0], labels[0])
print("\n")
print("The dataset contain: ",len(sens), "Sentences")
print("The dataset contain: ",len(labels), "Labels")
print("The dataset contain: ",len(words), "Unique Token")

Sentence and labels:

['RemarkThe', 'purely', 'radiative', 'spacetimes', 'used', 'as', 'reference', 'solutions', 'in', 'our', 'analysis', 'are', 'not', 'perturbations', 'of', 'the', 'Minkowski', 'spacetime.', 'A', 'way', 'of', 'seeing', 'this', 'is', 'to', 'consider', 'the', 'Newman–Penrose', 'constants', 'of', 'the', 'spacetime.', 'The', 'Newman–Penrose', 'constants', 'are', 'a', 'set', 'of', 'absolutely', 'conserved', 'quantities', 'defined', 'as', 'integrals', 'of', 'certain', 'components', 'of', 'the', 'Weyl', 'tensor', 'and', 'the', 'Maxwell', 'fields', 'over', 'cuts', 'of', 'null', 'infinity—see', '[', '19–21', ']', 'for', 'the', 'Einstein–Maxwell', 'case.', 'In', '[', '22', ']', 'it', 'has', 'been', 'shown', 'that', 'the', 'value', 'of', 'the', 'Newman–Penrose', 'constants', 'for', 'a', 'vacuum', 'radiative', 'spacetime', 'coincides', 'with', 'the', 'value', 'of', 'the', 'rescaled', 'Weyl', 'spinor', 'at', 'i+—this', 'result', 'can', 'be', 'extended', 'to', 'the', 'electrovacuum




3. creating dictonary with unique words and labels

In [31]:
#creating vocabulary with unique words and labels
word_dict = dict((c, i) for i, c in enumerate(words))
label_dict = dict((c, i) for i, c in enumerate(tags))

4. following one hot encoding method encodeded all sentences and labels

In [32]:
#spiliting into sents/labels
#spilitng into tokens
#append token value accoring to word vocabulary
#merging all tokens value in a new list
sens_to_int=[]
for i in sens:
    s_token_to_int=[]
    for j in i:
        s_token_to_int.append(word_dict[j])
    sens_to_int.append(s_token_to_int)

labels_to_int=[]
for i in labels:
    l_token_to_int=[]
    for j in i:
        l_token_to_int.append(label_dict[j])
    labels_to_int.append(l_token_to_int)

In [33]:
#visualize form of sentences and labels after encoding
print(sens_to_int[0], labels_to_int[0])

[3431, 3273, 6427, 1348, 5143, 6317, 560, 6932, 7760, 3942, 5966, 3772, 5596, 2992, 7464, 9952, 2757, 9247, 6974, 5113, 7464, 4653, 6240, 1846, 7535, 416, 9952, 4255, 4115, 7464, 9952, 9247, 8539, 4255, 4115, 3772, 6585, 2159, 7464, 6590, 1099, 5724, 5652, 6317, 3560, 7464, 8887, 525, 7464, 9952, 4412, 5151, 8044, 9952, 5906, 6627, 8198, 4167, 7464, 3558, 9258, 9215, 4078, 6001, 8060, 9952, 4680, 2167, 3979, 9215, 5640, 6001, 7660, 1261, 1626, 4369, 3812, 9952, 8623, 7464, 9952, 4255, 4115, 8060, 6585, 3052, 6427, 1436, 4262, 4616, 9952, 8623, 7464, 9952, 7288, 4412, 5064, 3713, 444, 7933, 5856, 1944, 6713, 7535, 9952, 2653, 5534, 7875, 9952, 2881, 7464, 6240, 9410, 4643, 9952, 6427, 1348, 2063, 9354, 9952, 9043, 7464, 9215, 3741, 6001, 7660, 5856, 1944, 5659, 3812, 9952, 8623, 7464, 9952, 4412, 5064, 3713, 9483, 1846, 8854, 9952, 512, 6880, 7464, 9952, 7472, 1193, 9247, 5277, 5092, 3984, 3812, 9952, 4255, 4115, 7464, 9952, 6427, 1436, 5856, 9407, 2411, 9107, 4207, 9952, 1016, 3525, 39

In [34]:
max_len_sens=max([len(i) for i in sens_to_int])#sent max length
max_len_labels=max([len(i) for i in labels_to_int])#labels max length
n_tags=len(tags)#number of tags in dataset
n_words=len(words)#number of words in vocabulary

4. Padding encoded sentences and labels with the maximum length of documents that confirmes they are in the same length

In [35]:
#first parameter is the sequence
#second parameter is the maximum length of sentence/label
#third parameter is position of padding is "post" or "pre"
#fouth parameter is the padding value
X = pad_sequences(sens_to_int, max_len_sens, padding="post",value=n_words - 1)
Y = pad_sequences(labels_to_int, max_len_labels, padding="post",value=label_dict["O"])

5. Categorizing labels with number of tags containing in dataset

In [36]:
y = [to_categorical(i, num_classes=n_tags) for i in Y]
y=np.array(y)

6. Spiliting dataset before feed into Neural Network

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [38]:
print(np.shape(X_train),np.shape(y_train))

(280, 318) (280, 318, 7)


In [39]:
print(type(X_train),type(y_train))

<class 'numpy.ndarray'> <class 'numpy.ndarray'>


5. Making artchitecture of model and here we use LSTM as a Neural Network

In [40]:
#sequential class
model = Sequential()
# Add an Embedding layer expecting input vocab of size,
# output embedding dimension of size 300, and
#input_length is a max length of sentences
model.add(Embedding(input_dim=n_words, output_dim=300, input_length=max_len_sens))

# LSTM layer with 512 internal memory units.
#input shape will be maximum length of sentence with n_tags values
model.add(LSTM(512, input_shape=(max_len_sens,n_tags), return_sequences=True))

# Add a Dense layer with number of tags.
#softmax activation function is used on the output to predict multinomial probability distribution.
model.add(TimeDistributed(Dense(n_tags, activation="softmax")))

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 318, 300)          3015600   
_________________________________________________________________
lstm_3 (LSTM)                (None, 318, 512)          1665024   
_________________________________________________________________
time_distributed_3 (TimeDist (None, 318, 7)            3591      
Total params: 4,684,215
Trainable params: 4,684,215
Non-trainable params: 0
_________________________________________________________________


In [41]:
#the multinomial log loss (categorical_crossentropy in Keras)
#The efficient ADAM optimization algorithm is used to find the weights
#the accuracy metric is calculated and reported each epoch
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

6. Feeding data into model

In [42]:
# fit model for 10 epoch on this sequence
history = model.fit(X_train, y_train, batch_size=32, epochs=10, validation_split=0.2, verbose=1)

Epoch 1/10
7/7 [==============================] - 28s 4s/step - loss: 1.6438 - accuracy: 0.5670 - val_loss: 0.9491 - val_accuracy: 0.8347
Epoch 2/10
7/7 [==============================] - 26s 4s/step - loss: 0.8157 - accuracy: 0.8399 - val_loss: 0.7124 - val_accuracy: 0.8347
Epoch 3/10
7/7 [==============================] - 33s 5s/step - loss: 0.7357 - accuracy: 0.8317 - val_loss: 0.7148 - val_accuracy: 0.8347
Epoch 4/10
7/7 [==============================] - 34s 5s/step - loss: 0.6844 - accuracy: 0.8376 - val_loss: 0.6801 - val_accuracy: 0.8347
Epoch 5/10
7/7 [==============================] - 35s 5s/step - loss: 0.6746 - accuracy: 0.8328 - val_loss: 0.6535 - val_accuracy: 0.8347
Epoch 6/10
7/7 [==============================] - 32s 5s/step - loss: 0.6166 - accuracy: 0.8391 - val_loss: 0.6286 - val_accuracy: 0.8347
Epoch 7/10
7/7 [==============================] - 31s 4s/step - loss: 0.5925 - accuracy: 0.8353 - val_loss: 0.6045 - val_accuracy: 0.8346
Epoch 8/10
7/7 [==================

7. Evaluate model

In [68]:
TP=FP=FN=TN=0
for j in range(len(X_test)):
    p = model.predict(np.array([X_test[j]]))
    y_pred = np.argmax(p, axis=-1)
    y_real=np.argmax([y_test[j]],axis=-1)
    
    for i in range(len(y_real[0])):
        if y_pred[0][i]==y_real[0][i]==1:
            TP+=1
        elif y_real[0][i]==1 and y_real[0][i]!=y_pred[0][i]:
            FP+=1
        elif y_real[0][i]==y_pred[0][i]==0:
            TN+=1
        elif y_pred[0][i]==0 and y_real[0][i]!=y_pred[0][i]:
            FN+=1

precision=(TP/(TP+FP))
recall=(TP/(TP+FN))
f1_score= (2*precision*recall)/(precision + recall)
print(precision, recall, f1_score)

0.9969328513528316 0.9927461139896373 0.99483507774711


8. Predict with model

In [26]:
i=random.randint(0,len(X_test))
p = model.predict(np.array([X_test[i]]))
p = np.argmax(p, axis=-1)  
print("{}   {}".format("Word", "Pred"))
for w,pred in zip(X_test[0],p[0]):
    print("{:14}: {}".format(words[w],tags[pred]))


Word   Pred
Similar       : O
numerical     : I-Process
oscillations  : I-Task
to            : I-Task
those         : I-Task
described     : I-Process
above         : O
also          : O
emerge        : O
in            : O
the           : O
ISPM          : O
when          : O
utilising     : O
classical     : O
IBM           : O
kernels       : O
due           : I-Process
to            : O
their         : O
lack          : O
of            : O
regularity    : O
(             : O
with          : O
discontinuous : O
second        : O
derivatives   : O
)             : O
.             : O
Furthermore   : O
,             : O
it            : O
is            : O
important     : O
to            : O
remark        : O
that          : O
the           : O
immersed      : O
structure     : O
stresses      : O
are           : O
captured      : O
in            : O
the           : O
Lagrangian    : O
description   : O
and           : O
hence         : O
,             : O
in            : O
order        